In [1]:
import tensorflow as tf
import numpy as np
import torch
import random
import pandas as pd
import os, sys
import time
import datetime
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
import operator
import statistics

# random.seed(42)
torch.manual_seed(42) 
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM3-32GB


########################### load model ###########################

In [2]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 8, # The number of output labels--2 for binary classification.  
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


model.cuda()


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

########################### Load dataset ###########################

In [3]:
import job_data_unbalanced_train
# import job_data_unbalanced_test
df = job_data_unbalanced_train.final_bios_df
# test_df = job_data_unbalanced_test.test_final_bios_df
df

,raw_title,gender,label,scrubbed,bio,swapped,prof
69206,Architect,M,1,_ is a graduate of South San High School and r...,He is a graduate of South San High School and ...,She is a graduate of South San High School and...,architect
43204,Nurse,F,13,_ has worked in rural health for many years an...,She has worked in rural health for many years ...,He has worked in rural health for many years a...,nurse
8744,Senior Software Engineer,F,24,"_ is a core maintainer of Kubernetes kops, the...","She is a core maintainer of Kubernetes kops, t...","He is a core maintainer of Kubernetes kops, th...",software_engineer
80641,model,F,12,_ has featured in various TV/editorial/commerc...,Ramona has featured in various TV/editorial/co...,_ has featured in various TV/editorial/commerc...,model
93619,plastic surgeon,M,25,"Born and raised in Florida, _ received _ under...","Born and raised in Florida, he received his un...","Born and raised in Florida, she received her u...",surgeon
...,...,...,...,...,...,...,...
86726,orthopedic surgeon,M,25,Dr. _ graduated summa cum laude from Tufts Uni...,Dr. Kolker graduated summa cum laude from Tuft...,Dr. Kolker graduated summa cum laude from Tuft...,surgeon
86319,dietitian,F,7,_ is the mother of 2 little boys and loves hel...,She is the mother of 2 little boys and loves h...,He is the mother of 2 little boys and loves he...,dietitian
93247,surgeon,M,25,_ specializes in endoscopy and laparoscopic an...,He specializes in endoscopy and laparoscopic a...,She specializes in endoscopy and laparoscopic ...,surgeon
28996,architect,F,1,_ passion for serving and guiding _ clients fr...,Her passion for serving and guiding her client...,His passion for serving and guiding his client...,architect


In [4]:
# import job_data
# df = job_data.classifier_df
# male_list = job_data.male_jobs
# male_list

In [5]:
df.rename(columns={'label':'old_label'}, inplace=True)
df

,raw_title,gender,old_label,scrubbed,bio,swapped,prof
69206,Architect,M,1,_ is a graduate of South San High School and r...,He is a graduate of South San High School and ...,She is a graduate of South San High School and...,architect
43204,Nurse,F,13,_ has worked in rural health for many years an...,She has worked in rural health for many years ...,He has worked in rural health for many years a...,nurse
8744,Senior Software Engineer,F,24,"_ is a core maintainer of Kubernetes kops, the...","She is a core maintainer of Kubernetes kops, t...","He is a core maintainer of Kubernetes kops, th...",software_engineer
80641,model,F,12,_ has featured in various TV/editorial/commerc...,Ramona has featured in various TV/editorial/co...,_ has featured in various TV/editorial/commerc...,model
93619,plastic surgeon,M,25,"Born and raised in Florida, _ received _ under...","Born and raised in Florida, he received his un...","Born and raised in Florida, she received her u...",surgeon
...,...,...,...,...,...,...,...
86726,orthopedic surgeon,M,25,Dr. _ graduated summa cum laude from Tufts Uni...,Dr. Kolker graduated summa cum laude from Tuft...,Dr. Kolker graduated summa cum laude from Tuft...,surgeon
86319,dietitian,F,7,_ is the mother of 2 little boys and loves hel...,She is the mother of 2 little boys and loves h...,He is the mother of 2 little boys and loves he...,dietitian
93247,surgeon,M,25,_ specializes in endoscopy and laparoscopic an...,He specializes in endoscopy and laparoscopic a...,She specializes in endoscopy and laparoscopic ...,surgeon
28996,architect,F,1,_ passion for serving and guiding _ clients fr...,Her passion for serving and guiding her client...,His passion for serving and guiding his client...,architect


In [6]:
label_dict = {'architect': 0,
 'surgeon': 1,
 'software_engineer': 2,
 'pastor': 3,
 'nurse': 4,
 'model': 5,
 'dietitian': 6,
 'paralegal': 7}

In [7]:
df.rename(columns={'label':'old_label'}, inplace=True)
df['label'] = df['prof'].map(label_dict)
df

,raw_title,gender,old_label,scrubbed,bio,swapped,prof,label
69206,Architect,M,1,_ is a graduate of South San High School and r...,He is a graduate of South San High School and ...,She is a graduate of South San High School and...,architect,0
43204,Nurse,F,13,_ has worked in rural health for many years an...,She has worked in rural health for many years ...,He has worked in rural health for many years a...,nurse,4
8744,Senior Software Engineer,F,24,"_ is a core maintainer of Kubernetes kops, the...","She is a core maintainer of Kubernetes kops, t...","He is a core maintainer of Kubernetes kops, th...",software_engineer,2
80641,model,F,12,_ has featured in various TV/editorial/commerc...,Ramona has featured in various TV/editorial/co...,_ has featured in various TV/editorial/commerc...,model,5
93619,plastic surgeon,M,25,"Born and raised in Florida, _ received _ under...","Born and raised in Florida, he received his un...","Born and raised in Florida, she received her u...",surgeon,1
...,...,...,...,...,...,...,...,...
86726,orthopedic surgeon,M,25,Dr. _ graduated summa cum laude from Tufts Uni...,Dr. Kolker graduated summa cum laude from Tuft...,Dr. Kolker graduated summa cum laude from Tuft...,surgeon,1
86319,dietitian,F,7,_ is the mother of 2 little boys and loves hel...,She is the mother of 2 little boys and loves h...,He is the mother of 2 little boys and loves he...,dietitian,6
93247,surgeon,M,25,_ specializes in endoscopy and laparoscopic an...,He specializes in endoscopy and laparoscopic a...,She specializes in endoscopy and laparoscopic ...,surgeon,1
28996,architect,F,1,_ passion for serving and guiding _ clients fr...,Her passion for serving and guiding her client...,His passion for serving and guiding his client...,architect,0


In [8]:
gender = {'M': 1,'F': 0}
df.gender = [gender[item] for item in df.gender]
df

,raw_title,gender,old_label,scrubbed,bio,swapped,prof,label
69206,Architect,1,1,_ is a graduate of South San High School and r...,He is a graduate of South San High School and ...,She is a graduate of South San High School and...,architect,0
43204,Nurse,0,13,_ has worked in rural health for many years an...,She has worked in rural health for many years ...,He has worked in rural health for many years a...,nurse,4
8744,Senior Software Engineer,0,24,"_ is a core maintainer of Kubernetes kops, the...","She is a core maintainer of Kubernetes kops, t...","He is a core maintainer of Kubernetes kops, th...",software_engineer,2
80641,model,0,12,_ has featured in various TV/editorial/commerc...,Ramona has featured in various TV/editorial/co...,_ has featured in various TV/editorial/commerc...,model,5
93619,plastic surgeon,1,25,"Born and raised in Florida, _ received _ under...","Born and raised in Florida, he received his un...","Born and raised in Florida, she received her u...",surgeon,1
...,...,...,...,...,...,...,...,...
86726,orthopedic surgeon,1,25,Dr. _ graduated summa cum laude from Tufts Uni...,Dr. Kolker graduated summa cum laude from Tuft...,Dr. Kolker graduated summa cum laude from Tuft...,surgeon,1
86319,dietitian,0,7,_ is the mother of 2 little boys and loves hel...,She is the mother of 2 little boys and loves h...,He is the mother of 2 little boys and loves he...,dietitian,6
93247,surgeon,1,25,_ specializes in endoscopy and laparoscopic an...,He specializes in endoscopy and laparoscopic a...,She specializes in endoscopy and laparoscopic ...,surgeon,1
28996,architect,0,1,_ passion for serving and guiding _ clients fr...,Her passion for serving and guiding her client...,His passion for serving and guiding his client...,architect,0


Data for training

In [9]:
# Get the lists of sentences and their labels.
sentences = df.bio.values
gender = df.gender.values
labels = df.label.values



# Split dataset into train, validation, test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, gen_train, gen_test = train_test_split(sentences, labels, gender, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val, gen_train, gen_val = train_test_split(X_train, y_train, gen_train, test_size=0.05, random_state=1)

########################### tokenize dataset ###########################

In [10]:
# from transformers import BertTokenizer, AdamW, BertConfig, BertForPreTraining

# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
seq_len = 100
vocab_length = 30522
def tokenize(data, y, g):
    

    # Load the BERT tokenizer.
    print('Loading BERT tokenizer...')

    # Training Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for k, sent in enumerate(data):
        encoded_dict = tokenizer.encode_plus(
                            str(sent),                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = seq_len,           # Pad & truncate all sentences.
                            truncation=True,
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
        

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        
        
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    class_labels = torch.tensor(y)
    gender_labels = torch.tensor(g)
    
    
    


#     print(y)
#     print(y_onehot)

    return input_ids, class_labels, attention_masks, gender_labels



In [11]:

train_input_ids, train_class_labels, train_attention_masks, train_gender_labels = tokenize(X_train, y_train, gen_train)
test_input_ids, test_class_labels, test_attention_masks, test_gender_labels = tokenize(X_test, y_test, gen_test)
val_input_ids, val_class_labels, val_attention_masks, val_gender_labels = tokenize(X_val, y_val, gen_val)

print('Original: ', X_train[0])
print('Token IDs:', train_input_ids[0])
print('Token IDs:', train_class_labels[0])




Loading BERT tokenizer...


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Loading BERT tokenizer...
Loading BERT tokenizer...
Original:  Dr. Lundeen graduated from University of Utah School of Medicine in 2000 and has been in practice for 18 years. He completed a residency at Carolinas Medical Center. Dr. Lundeen also specializes in Foot & Ankle Surgery. He currently practices at Reno Orthopaedic Clinic and is affiliated with Northern Nevada Medical Center. Dr. Lundeen accepts multiple insurance plans including Aetna, Hometown Health and Health Plan of Nevada. In addition to English, Dr. Lundeen's practice supports this language: Spanish.
Token IDs: tensor([  101,  2852,  1012, 21860, 12129,  3852,  2013,  2118,  1997,  6646,
         2082,  1997,  4200,  1999,  2456,  1998,  2038,  2042,  1999,  3218,
         2005,  2324,  2086,  1012,  2002,  2949,  1037, 14079,  2012,  3792,
         2015,  2966,  2415,  1012,  2852,  1012, 21860, 12129,  2036, 16997,
         1999,  3329,  1004, 10792,  5970,  1012,  2002,  2747,  6078,  2012,
        17738,  2030,  270

In [12]:

from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_input_ids, train_class_labels, train_attention_masks, train_gender_labels)
val_dataset = TensorDataset(val_input_ids, val_class_labels, val_attention_masks, val_gender_labels)
test_dataset = TensorDataset(test_input_ids, test_class_labels, test_attention_masks, test_gender_labels)

print('{:>5,} training samples'.format(len(train_dataset)))
print('{:>5,} validation samples'.format(len(val_dataset)))
print('{:>5,} validation samples'.format(len(test_dataset)))

11,928 training samples
  628 validation samples
1,396 validation samples


########################### save dataset ###########################

In [13]:
# train_dataset = torch.load('unbalanced_datasets_bio_classification_train_dataset.pt')
# val_dataset = torch.load('unbalanced_datasets_bio_classification_val_dataset.pt')
# test_dataset = torch.load('unbalanced_datasets_bio_classification_test_dataset.pt')

########################### Load dataset ###########################

In [14]:
seq_len = 100
vocab_length = 30522

import time
import datetime

from transformers.optimization import AdamW
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )



from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


from transformers.optimization import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 8

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)




def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# import numpy as np


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


import random
from tqdm.notebook import tqdm

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
def train(model, optimizer, losslogger, start_epoch, epochs, run_id, train_dataloader, validation_dataloader, checkpoint_name):
    # Set the seed value all over the place to make this reproducible.
    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    t = tqdm(range(start_epoch,epochs))

    for epoch_i in t:

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.classifier.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
#             print(batch[0].shape)
#             print(batch[1].shape)
#             print(batch[2].shape)
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[2].to(device)
            b_labels = batch[1].to(device)
#             gen_labels = batch[3]
            
#             #     convert input ids to one hot encoding
#             b_input_ids = b_input_ids.reshape(len(b_input_ids), seq_len, 1)
#             y_onehot = torch.FloatTensor(len(b_input_ids), seq_len, vocab_length)
#             y_onehot.zero_()
#             y_onehot.scatter_(2, b_input_ids, 1)
#             y_onehot = y_onehot.float().to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            outputs = model(b_input_ids, 
#                                  token_type_ids=None, 
                                 attention_mask=b_input_mask, 
                                 labels=b_labels,
#                                  training_type="fine_tuning"
                                )

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_train_loss += outputs.loss.item()

            # Perform a backward pass to calculate the gradients.
            outputs.loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)            

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)
        
        # set in logs
        df = pd.DataFrame()
        df['chackpoint_name'] = pd.Series(checkpoint_name)
        df['epoch'] = pd.Series(epoch_i)
        df['Loss'] = pd.Series(outputs.loss.data.item())
        df['run'] = run_id
        losslogger = losslogger.append(df)
        
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))
        state = {'epoch': epoch_i + 1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'losslogger': losslogger, }
#         torch.save(state, f'{checkpoint_name}')

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using 
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[2].to(device)
            b_labels = batch[1].to(device)
            gen_labels = batch[3]
            
            
#             #     convert input ids to one hot encoding
#             b_input_ids = b_input_ids.reshape(len(b_input_ids), seq_len, 1)
#             y_onehot = torch.FloatTensor(len(b_input_ids), seq_len, vocab_length)
#             y_onehot.zero_()
#             y_onehot.scatter_(2, b_input_ids, 1)
#             y_onehot = y_onehot.float().to(device)
            
            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                outputs = model(b_input_ids, 
#                                        token_type_ids=None, 
                                       attention_mask=b_input_mask,
                                       labels=b_labels,
#                                        training_type="fine_tuning"
                                      )

            # Accumulate the validation loss.
            total_eval_loss += outputs.loss.item()

            # Move logits and labels to CPU
            logits = outputs.logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            gen_ids = gen_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, label_ids, )


        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

#     print("")
#     print("Training complete!")

#     print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
          
def load_checkpoint(model, optimizer, losslogger, filename):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        losslogger = checkpoint['losslogger']
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch, losslogger          
          
          
          
          


########################### Start training with checkpoint ###########################

In [15]:
# first training epoch
# Start
start_epoch = 0

# Logger
losslogger = pd.DataFrame()

# Checkpoint name
checkpoint_name = 'checkpoint.pth.tar'

train(model, optimizer, losslogger, start_epoch, epochs, 0, train_dataloader, validation_dataloader, checkpoint_name)
time.sleep(8)


======== Epoch 1 / 8 ========
Training...
  Batch    40  of    373.    Elapsed: 0:00:05.
  Batch    80  of    373.    Elapsed: 0:00:09.
  Batch   120  of    373.    Elapsed: 0:00:14.
  Batch   160  of    373.    Elapsed: 0:00:20.
  Batch   200  of    373.    Elapsed: 0:00:26.
  Batch   240  of    373.    Elapsed: 0:00:31.
  Batch   280  of    373.    Elapsed: 0:00:37.
  Batch   320  of    373.    Elapsed: 0:00:43.
  Batch   360  of    373.    Elapsed: 0:00:48.

  Average training loss: 0.61
  Training epcoh took: 0:00:50

Running Validation...
  Accuracy: 0.88
  Validation Loss: 0.32
  Validation took: 0:00:01

======== Epoch 2 / 8 ========
Training...
  Batch    40  of    373.    Elapsed: 0:00:05.
  Batch    80  of    373.    Elapsed: 0:00:11.
  Batch   120  of    373.    Elapsed: 0:00:17.
  Batch   160  of    373.    Elapsed: 0:00:23.
  Batch   200  of    373.    Elapsed: 0:00:29.
  Batch   240  of    373.    Elapsed: 0:00:35.
  Batch   280  of    373.    Elapsed: 0:00:41.
  Batch  

In [16]:
torch.save(model, "4_distilbert_base_uncased_classification.pth")

################## evaluate ###########################

In [18]:
batch_size = 32
import time
import datetime

from transformers.optimization import AdamW
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )



from torch.utils.data import DataLoader, RandomSampler, SequentialSampler



from scipy.special import softmax

# For validation the order doesn't matter, so we'll just read them sequentially.
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)



# ========================================
#               Testing
# ========================================
# After the completion of each training epoch, measure our performance on
# our validation set.

print("")
print("Running test...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_test_prob = []
actual_label_list = []
gen_label_list = []
pred_label_list = []
# Evaluate data for one epoch
for batch in test_dataloader:

    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[2].to(device)
    b_labels = batch[1].to(device)
    gen_labels = batch[3]


#             #     convert input ids to one hot encoding
#             b_input_ids = b_input_ids.reshape(len(b_input_ids), seq_len, 1)
#             y_onehot = torch.FloatTensor(len(b_input_ids), seq_len, vocab_length)
#             y_onehot.zero_()
#             y_onehot.scatter_(2, b_input_ids, 1)
#             y_onehot = y_onehot.float().to(device)

    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        outputs = model(b_input_ids, 
#                                token_type_ids=None, 
                               attention_mask=b_input_mask,
                               labels=b_labels,
#                                        training_type="fine_tuning"
                              )

    # Accumulate the validation loss.
    total_eval_loss += outputs.loss.item()

    # Move logits and labels to CPU
    logits = outputs.logits.detach().cpu().numpy()
    probs = softmax(logits, axis=1)
    pred_flat = list(np.argmax(logits, axis=1).flatten())
    label_ids = list(b_labels.to('cpu').numpy())
    label_ids2 = b_labels.to('cpu').numpy()
    gen_ids = list(gen_labels.to('cpu').numpy())
    
    for ind, lab in enumerate(label_ids):
#         print(probs[ind][lab])
        y_test_prob.append(probs[ind][lab])
    
    
    actual_label_list = (actual_label_list + label_ids)
    pred_label_list = (pred_label_list + pred_flat)
    gen_label_list = (gen_label_list + gen_ids)

    

#     actual_label_list.append(pred_flat)
#     gen_label_list.append(gen_ids)
#     pred_label_list.append(label_ids)
#     # Calculate the accuracy for this batch of test sentences, and
#     # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids2)

#     print(label_ids.shape)
# print(actual_label_list)
# print(gen_label_list)
# print(pred_label_list)

print(len(actual_label_list))
print(len(gen_label_list))
print(len(pred_label_list))

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# # Calculate the average loss over all of the batches.
# avg_val_loss = total_eval_loss / len(test_dataloader)

# # Measure how long the validation run took.
# validation_time = format_time(time.time() - t0)

# print("  Validation Loss: {0:.2f}".format(avg_val_loss))
# print("  Validation took: {:}".format(validation_time))

# # Record all statistics from this epoch.
# training_stats.append(
#     {
#         'epoch': epoch_i + 1,
#         'Training Loss': avg_train_loss,
#         'Valid. Loss': avg_val_loss,
#         'Valid. Accur.': avg_val_accuracy,
#         'Training Time': training_time,
#         'Validation Time': validation_time
#     }
# )


Running test...
1396
1396
1396
  Accuracy: 0.88


In [19]:
import importlib
import evaluation
importlib.reload(evaluation)

# evaluation.reload(eval)
evaluate = evaluation.eval(14, pred_label_list, actual_label_list, gen_label_list)


In [20]:
import collections

class_list = list(range(14))
class_list

# Function to calculate the demographic parity
# def flat_accuracy(preds, labels, gen):
preds = pred_label_list
labels = actual_label_list
gender = gen_label_list

job_0 = []
job_1 = []
job_2 = []
job_3 = []
job_4 = []
job_5 = []
job_6 = []
job_7 = []


class_dict = {0:job_0, 1:job_1, 2:job_2, 3:job_3, 4:job_4, 5:job_5, 6:job_6, 7:job_7}

for i in class_list:
    for ind, label in enumerate(labels):
        if label == i:
#             print(i,preds[ind])
            class_dict[i].append((labels[ind], preds[ind], gender[ind]))
# class_dict        

In [21]:
# TPR_diff scores
TPR_diff_list = []
for i in class_dict:
    TPR_diff_list.append(abs(evaluate.TPR_diff(i)))
print("average TPR_diff:", statistics.mean(TPR_diff_list), min(TPR_diff_list), max(TPR_diff_list))

0
1
2
3
4
5
6
7
average TPR_diff: 0.12655497261847165 0.010123119015047921 0.3178082191780822
